## Potential Talent

### **Context:**

As a **talent sourcing and management company**, we are interested in **finding talented individuals** for sourcing these candidates to technology companies. **Finding talented candidates is not easy**, for **several reasons**. The **first** reason is one needs to understand what the role is very well to fill in that spot, this requires understanding the client’s needs and what they are looking for in a potential candidate. The **second** reason is one needs to understand what makes a candidate shine for the role we are in search for. **Third**, where to find talented individuals is another challenge.

The nature of our job requires a lot of human labor and is full of **manual operations**. Towards **automating this process** we want to build a better approach that could save us time and finally help us spot potential candidates that could fit the roles we are in search for. Moreover, going beyond that for a specific role we want to fill in we are interested in developing a machine learning powered pipeline that could spot talented individuals, and rank them based on their fitness.

We are right now semi-automatically sourcing a few candidates, therefore the sourcing part is not a concern at this time but we expect to first determine best matching candidates based on how fit these candidates are for a given role. We generally make these searches based on some keywords such as “full-stack software engineer”, “engineering manager” or “aspiring human resources” based on the role we are trying to fill in. These keywords might change, and you can expect that specific keywords will be provided to you.

Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

### Data Description:

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

#### Attributes:
**id** : unique identifier for candidate (numeric)

**job_title** : job title for candidate (text)

**location** : geographical location for candidate (text)

**connections** : number of connections candidate has, 500+ means over 500 (text)

**Output (desired target)**:
fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”

#### Download Data:

https://docs.google.com/spreadsheets/d/117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU/edit?usp=sharing

#### Goal(s):

Predict how fit the candidate is based on their available information (variable fit)

Success Metric(s):

Rank candidates based on a fitness score.

Re-rank candidates when a candidate is starred.

#### Bonus(es):

We are interested in a robust algorithm, tell us how your solution works and show us how your ranking gets better with each starring action.

How can we filter out candidates which in the first place should not be in this list?

Can we determine a cut-off point that would work for other roles without losing high potential candidates?

Do you have any ideas that we should explore so that we can even automate this procedure to prevent human bias?

In [1]:
# !pip install -U scikit-learn

In [2]:
# Importing Standard Libraries
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import linear_kernel
pd.options.display.max_columns = 30

## Initial Exploratory Data Analysis

In [3]:
df = pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv').set_index('id')
df.head()

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 1 to 104
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   job_title   104 non-null    object 
 1   location    104 non-null    object 
 2   connection  104 non-null    object 
 3   fit         0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 4.1+ KB


In [5]:
df.replace('500+ ','501', inplace=True)

In [6]:
df['connection'] = pd.to_numeric(df['connection'])

In [7]:
df.job_title.value_counts()

Aspiring Human Resources Professional                                                                                    7
Student at Humber College and Aspiring Human Resources Generalist                                                        7
2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional                 7
People Development Coordinator at Ryan                                                                                   6
HR Senior Specialist                                                                                                     5
Native English Teacher at EPIK (English Program in Korea)                                                                5
Aspiring Human Resources Specialist                                                                                      5
SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR              4
Seeking Human Re

In [8]:
df.job_title.iloc[0]

'2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional'

In [9]:
df = df.drop_duplicates()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1 to 104
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   job_title   53 non-null     object 
 1   location    53 non-null     object 
 2   connection  53 non-null     int64  
 3   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.1+ KB


# TF-IDF

### Prepping our Text for Modelling

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Prep our Text for Modelling
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (1, 2))
docs_tfidf = vectorizer.fit_transform(df["job_title"])

In [12]:
def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query_tfidf = vectorizer.transform([query])
    cos_sim = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    
    return cos_sim

In [13]:
query = 'Aspiring human resources'

cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query)

df['fit'] = cos_sim

In [14]:
def top_candidates(n, by = 'fit', ascending = False, min_con = 0, location = df.location):
    
    df2 = df.loc[(df.connection >= min_con) & (df.location == location)].sort_values(by = by, ascending = ascending).head(n).copy()
    
    return df2

In [15]:
top_candidates(n = 10, by = 'fit', ascending = False, min_con = 0)

,job_title,location,connection,fit
id,,,,
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.735855
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.735855
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.632697
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.508880
72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,0.387590
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",501,0.374733
66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.373847
7,Student at Humber College and Aspiring Human R...,Kanada,61,0.358949
74,Human Resources Professional,Greater Boston Area,16,0.340769


In [16]:
top_candidates(n = 10, by = 'fit', ascending = False, min_con = 90)

,job_title,location,connection,fit
id,,,,
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",501,0.374733
82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.316420
100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.308829
76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.246772
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.220668
101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",501,0.196509
78,Human Resources Generalist at Schwan's,Amerika Birleşik Devletleri,501,0.196509
71,"Human Resources Generalist at ScottMadden, Inc.","Raleigh-Durham, North Carolina Area",501,0.196509
68,Human Resources Specialist at Luxottica,Greater New York City Area,501,0.189503


In [17]:
top_candidates(n = 50, by = 'fit', ascending = False, location = 'Austin, Texas Area')

,job_title,location,connection,fit
id,,,,
66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.373847
82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.316420


In [18]:
top_candidates(n = 50, by = 'fit', ascending = False, location = 'Greater New York City Area')

,job_title,location,connection,fit
id,,,,
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.632697
68,Human Resources Specialist at Luxottica,Greater New York City Area,501,0.189503
102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,0.000000


In [19]:
query = 'seeking human resources'

cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query)

df['fit'] = cos_sim

In [20]:
top_candidates(n = 10, by = 'fit', ascending = False, min_con = 0)

,job_title,location,connection,fit
id,,,,
99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.675682
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.675682
10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,501,0.432761
94,Seeking Human Resources Opportunities. Open t...,Amerika Birleşik Devletleri,415,0.381290
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.362648
74,Human Resources Professional,Greater Boston Area,16,0.295223
75,"Nortia Staffing is seeking Human Resources, Pa...","San Jose, California",501,0.273577
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",501,0.245337
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.240319


# Word2Vec

In [21]:
# !pip install nltk
# !pip install keras
# !pip install -U gensim

In [22]:
# !pip install tensorflow

In [23]:
import tensorflow as tf

### Prepping our Text for Modelling

In [24]:
import re
import nltk
# nltk.download('stopwords')

# processing texts for modelling
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df['job_title_cleaned'] = df.job_title.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() 
                                                                                  for w in x.split() 
                                                                                  if re.sub(r'[^a-zA-Z]',' ',w).lower() 
                                                                                  not in stop_words) )

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1 to 104
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          53 non-null     object 
 1   location           53 non-null     object 
 2   connection         53 non-null     int64  
 3   fit                53 non-null     float64
 4   job_title_cleaned  53 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.5+ KB


In [26]:
# tokenize and pad every document to make them of the same size
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()

tokenizer.fit_on_texts(df.job_title_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(df.job_title_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1

In [27]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim
W2V_PATH="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

In [28]:
model_w2v[0][0]

0.0011291504

In [29]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]
        
# creating document-word embeddings
document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))
for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
document_word_embeddings.shape

(53, 64, 300)

In [30]:
document_word_embeddings[0][0]

array([-2.08007812e-01,  3.41796875e-02,  2.57568359e-02,  1.79687500e-01,
       -1.81640625e-01, -3.41796875e-02, -1.40625000e-01, -1.63085938e-01,
       -8.59375000e-02, -1.52343750e-01, -9.57031250e-02, -1.34765625e-01,
       -1.92382812e-01,  2.43164062e-01, -1.91406250e-01,  4.93164062e-02,
        2.60009766e-02,  3.28125000e-01, -7.37304688e-02,  5.05371094e-02,
       -1.52343750e-01, -1.57226562e-01, -1.44958496e-04, -2.51953125e-01,
       -4.22363281e-02, -1.72119141e-02, -4.84375000e-01,  2.07031250e-01,
       -1.40625000e-01, -1.35498047e-02, -1.78222656e-02,  5.95092773e-03,
       -3.10058594e-02, -2.75390625e-01, -2.65625000e-01,  9.52148438e-02,
       -4.55078125e-01,  1.13281250e-01, -1.33789062e-01,  1.18652344e-01,
       -5.37109375e-02,  8.10546875e-02,  7.32421875e-02,  6.39648438e-02,
       -9.47265625e-02,  4.39453125e-02,  1.46484375e-01, -8.59375000e-02,
       -1.58203125e-01,  1.63085938e-01, -1.32812500e-01,  2.50000000e-01,
       -5.61523438e-02,  

In [31]:
# cosine_similarity = np.dot(model_w2v['spain'], model_w2v['england'])/(np.linalg.norm(model_w2v['spain'])* 
#                                                                       np.linalg.norm(model_w2v['england']))
# cosine_similarity

In [32]:
model_w2v['england'][:5]

array([-0.3671875 , -0.03491211,  0.11083984,  0.40039062,  0.18261719],
      dtype=float32)

In [33]:
def processing(query):
    df3 = pd.DataFrame([query], columns=['query'])
    stop_words = stopwords.words('english')
    df3['processed'] = df3['query'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() 
                                                                                  for w in x.split() 
                                                                                  if re.sub(r'[^a-zA-Z]',' ',w).lower() 
                                                                                  not in stop_words) )
    
    tokenizer.fit_on_texts(df3.processed)
    tokenized_documents=tokenizer.texts_to_sequences(df3.processed)
    tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
    vocab_size=len(tokenizer.word_index)+1
    
    embedding_matrix=np.zeros((vocab_size,300))
    for word,i in tokenizer.word_index.items():
        if word in model_w2v:
            embedding_matrix[i]=model_w2v[word]

    # creating document-word embeddings
    query_document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))
    for i in range(len(tokenized_paded_documents)):
        for j in range(len(tokenized_paded_documents[0])):
            query_document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
#     document_word_embeddings.shape
    
    return query_document_word_embeddings

In [34]:
processing('hello world!!!!').shape

(1, 64, 300)

In [35]:
processing('hello world!!!!')[0][:3][0][:20]

array([-0.05419922,  0.01708984, -0.00527954,  0.33203125, -0.25      ,
       -0.01397705, -0.15039062, -0.265625  ,  0.01647949,  0.3828125 ,
       -0.03295898, -0.09716797, -0.16308594, -0.04443359,  0.00946045,
        0.18457031,  0.03637695,  0.16601562,  0.36328125, -0.25585938])

In [36]:
def get_w2v_query_similarity(document_word_embeddings, query):
    """
    query_w2v: processing the query
    model_w2v: word2vec embedding for all docs
    query: query doc

    return: cosine similarity between query and all docs

    """
    query_w2v = processing(query)
    
    nsamples, nx, ny = query_w2v.shape
    query_w2v_reshape = query_w2v.reshape((nsamples,nx*ny))

    nsamples, nx, ny = document_word_embeddings.shape
    document_word_embeddings_reshape = document_word_embeddings.reshape((nsamples,nx*ny))
    
    cos_sim_w2v = cosine_similarity(query_w2v_reshape, document_word_embeddings_reshape).flatten()
    
    return cos_sim_w2v

In [37]:
query = 'Aspiring human resources'

# Word2Vec Similarity
cos_sim_w2v = get_w2v_query_similarity(document_word_embeddings, query = query)
df['w2v_fit'] = cos_sim_w2v

# Original TFIDF similarity for comparison
cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query) 
df['tfidf_fit'] = cos_sim
# Dropping the original fit column
df.drop('fit', axis=1, inplace=True)

In [38]:
top_candidates(n = 10, by = 'w2v_fit', ascending = False, min_con = 0)

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit
id,,,,,,
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,0.898174,0.735855
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,0.898174,0.735855
6,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,0.873679,0.632697
99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,seeking human resources position,0.654387,0.220668
82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,aspiring human resources professional energe...,0.641739,0.316420
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",501,aspiring human resources management student se...,0.628601,0.374733
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,seeking human resources opportunities,0.619797,0.220668
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seeking inte...,0.584569,0.508880
76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,aspiring human resources professional passio...,0.551164,0.246772


In [39]:
query = 'seeking human resources'

# Word2Vec Similarity
cos_sim_w2v = get_w2v_query_similarity(document_word_embeddings, query = query)
df['w2v_fit'] = cos_sim_w2v

# Original TFIDF similarity for comparison
cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query) 
df['tfidf_fit'] = cos_sim

In [40]:
top_candidates(n = 10, by = 'w2v_fit', ascending = False, min_con = 0)

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit
id,,,,,,
99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,seeking human resources position,0.886226,0.675682
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,seeking human resources opportunities,0.839381,0.675682
10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,501,seeking human resources hris generalist positions,0.703341,0.432761
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,0.663209,0.240319
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,0.663209,0.240319
6,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,0.645122,0.206629
94,Seeking Human Resources Opportunities. Open t...,Amerika Birleşik Devletleri,415,seeking human resources opportunities open tr...,0.639099,0.381290
89,Director Human Resources at EY,Greater Atlanta Area,349,director human resources ey,0.571728,0.162381
82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,aspiring human resources professional energe...,0.473859,0.103338


In [41]:
query = 'business intelligence specialist'

# Word2Vec Similarity
cos_sim_w2v = get_w2v_query_similarity(document_word_embeddings, query = query)
df['w2v_fit'] = cos_sim_w2v

# Original TFIDF similarity for comparison
cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query) 
df['tfidf_fit'] = cos_sim

In [42]:
top_candidates(n = 10, by = 'w2v_fit', ascending = False, min_con = 0)

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit
id,,,,,,
102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,business intelligence analytics travelers,0.552532,0.560060
68,Human Resources Specialist at Luxottica,Greater New York City Area,501,human resources specialist luxottica,0.447380,0.178214
8,HR Senior Specialist,San Francisco Bay Area,501,hr senior specialist,0.348536,0.168359
86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,information systems specialist programmer love...,0.274835,0.099972
101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",501,human resources generalist loparex,0.251939,0.000000
78,Human Resources Generalist at Schwan's,Amerika Birleşik Devletleri,501,human resources generalist schwan s,0.231181,0.000000
13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",501,human resources coordinator intercontinental b...,0.215800,0.000000
72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,business management major aspiring human resou...,0.214225,0.122980
4,People Development Coordinator at Ryan,"Denton, Texas",501,people development coordinator ryan,0.205907,0.000000


In [43]:
top_candidates(n = 10, by = 'w2v_fit', ascending = False, min_con = 20, location = 'Greater New York City Area')

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit
id,,,,,,
102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,business intelligence analytics travelers,0.552532,0.560060
68,Human Resources Specialist at Luxottica,Greater New York City Area,501,human resources specialist luxottica,0.447380,0.178214


# GloVe - 

https://nlp.stanford.edu/projects/glove/

In [44]:
# !pip install wget

In [45]:
# Downloading GloVe pre-trained vectors
# import wget
# wget.download('https://nlp.stanford.edu/data/glove.840B.300d.zip')

In [46]:
# Extracting GloVe vector file
# import zipfile as zf
# files = zf.ZipFile("glove.840B.300d.zip", 'r')
# files.extractall('GloVe')
# files.close()

In [47]:
path = 'GloVe/glove.840B.300d.txt'

In [48]:
with open(path) as file:
  for i in range(10):
    line = file.readline()
    print(line[:100])

, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0
. 0.012001 0.20751 -0.12578 -0.59325 0.12525 0.15975 0.13748 -0.33157 -0.13694 1.7893 -0.47094 0.704
the 0.27204 -0.06203 -0.1884 0.023225 -0.018158 0.0067192 -0.13877 0.17708 0.17709 2.5882 -0.35179 -
and -0.18567 0.066008 -0.25209 -0.11725 0.26513 0.064908 0.12291 -0.093979 0.024321 2.4926 -0.017916
to 0.31924 0.06316 -0.27858 0.2612 0.079248 -0.21462 -0.10495 0.15495 -0.03353 2.4834 -0.50904 0.087
of 0.060216 0.21799 -0.04249 -0.38618 -0.15388 0.034635 0.22243 0.21718 0.0068483 2.4375 -0.27418 0.
a 0.043798 0.024779 -0.20937 0.49745 0.36019 -0.37503 -0.052078 -0.60555 0.036744 2.2085 -0.23389 -0
in 0.089187 0.25792 0.26282 -0.029365 0.47187 -0.10389 -0.10013 0.08123 0.20883 2.5726 -0.67854 0.03
" -0.075242 0.57337 -0.31908 -0.18484 0.88867 -0.27381 0.077588 0.13905 -0.47746 1.4442 -0.56159 0.0
: 0.008746 0.33214 -0.29175 -0.15119 -0.41842 -0.23931 -0.23458 -0.055618 -0.09896 0.75175 

In [49]:
df_glove = pd.read_csv(path, sep=" ", quoting=3, header=None, index_col=0)
df_glove.T

,",",.,the,and,to,of,a,in,"""",:,is,for,I,),(,...,what-might-have-been,wiid,windowsTransgender,woombie,wordsforyoungmen,work.Like,working.So,wried,wwent,xalisae,xtremecaffeine,yildirim,z/28,zipout,zulchzulu
1,-0.082752,0.012001,0.272040,-0.185670,0.319240,0.060216,0.043798,0.089187,-0.075242,0.008746,-0.084961,-0.172240,0.194100,-0.271420,-0.180240,...,0.562950,0.385100,-0.102350,0.65711,-0.378200,-0.23822,0.754650,0.54698,0.921790,0.337540,0.073032,0.222760,0.73440,0.21215,-0.079690
2,0.672040,0.207510,-0.062030,0.066008,0.063160,0.217990,0.024779,0.257920,0.573370,0.332140,0.502000,0.182340,0.226030,0.047374,0.008411,...,-0.293780,-0.315230,-0.043862,-1.06710,-1.154600,-0.65700,-0.292360,-0.50515,-0.344320,-0.131110,-1.029400,-0.296390,-0.33641,-0.99456,-0.229050
3,-0.149870,-0.125780,-0.188400,-0.252090,-0.278580,-0.042490,-0.209370,0.262820,-0.319080,-0.291750,0.002382,-0.278470,-0.437640,-0.172780,-0.304630,...,1.279200,-0.103690,0.193270,-0.27787,0.387420,-0.18234,-0.211210,0.56164,-0.508880,0.155950,-0.015436,0.694120,0.26918,1.17820,0.803660
4,-0.064983,-0.593250,0.023225,-0.117250,0.261200,-0.386180,0.497450,-0.029365,-0.184840,-0.151190,-0.167550,-0.084666,-0.113870,-0.029084,0.209970,...,-0.070849,-0.110770,-0.225560,0.48507,1.288100,-0.27082,-0.105820,-0.29412,-0.000386,-1.124400,0.726150,0.193620,0.41843,2.07210,-0.788650
5,0.056491,0.125250,-0.018158,0.265130,0.079248,-0.153880,0.360190,0.471870,0.888670,-0.418420,0.307210,0.254420,-0.072725,-0.219100,0.085153,...,-0.487520,0.172950,-0.148480,-0.51166,-0.802670,0.37388,-0.295270,-0.35497,-0.151450,0.100460,-0.992460,-0.312760,-0.18900,-0.44271,-0.405670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.053380,0.063500,-0.018168,-0.039709,-0.258100,0.329200,0.080421,0.193680,-0.212800,0.700590,0.275500,-0.122380,-0.036109,0.066062,0.539500,...,-0.080746,0.092665,-0.223050,-0.43677,0.503820,0.28653,-0.484500,-0.21834,-0.406440,-0.094080,0.127380,-0.413980,-0.48432,-0.41382,-0.205220
297,-0.050821,0.140190,0.114070,0.324980,-0.044629,-0.175970,-0.061246,-0.325460,-0.226150,-0.213710,-0.067180,-0.081083,0.112210,-0.241770,-0.235960,...,-0.571630,-0.087504,0.161510,-0.43438,0.067534,-0.83764,-0.343440,0.44104,0.119900,-0.361960,0.129570,0.240180,-1.00550,-0.21139,0.268780
298,-0.191800,0.138710,0.130150,-0.023452,0.082745,0.117090,-0.300990,0.144210,0.328000,-0.286770,-0.215110,-0.126680,0.091957,-0.319120,-0.385520,...,0.428840,0.822160,0.119740,0.28584,0.852120,0.35327,0.935510,0.84572,0.380180,0.645910,0.422640,0.093864,0.63718,0.93427,-0.083561
299,-0.378460,-0.360490,-0.183170,0.123020,0.097801,-0.166920,-0.145840,-0.169000,-0.109340,-0.226630,-0.263040,-0.438560,0.386320,0.235390,0.243240,...,-0.078753,-0.591150,0.290980,-0.18967,-0.973490,0.13764,-0.099674,-0.78417,0.345800,-0.080984,-0.113330,-0.165450,-0.13914,-0.93286,0.485320


In [50]:
glove = { key: val.values for key, val in df_glove.T.items() }

In [51]:
glove['man'][:20]

array([-1.7310e-01,  2.0663e-01,  1.6543e-02, -3.1026e-01,  1.9719e-02,
        2.7791e-01,  1.2283e-01, -2.6328e-01,  1.2522e-01,  3.1894e+00,
       -1.6291e-01, -8.8759e-02,  3.3067e-03, -2.9483e-03, -3.4398e-01,
        1.2779e-01, -9.4536e-02,  4.3467e-01,  4.9742e-01,  2.5068e-01])

In [52]:
unknown_word = df_glove.mean().values
unknown_word[:20]

array([ 0.22418612, -0.28881808,  0.13854355,  0.00365397, -0.12870769,
        0.1024395 ,  0.06162703,  0.07317769, -0.06135387, -1.34764119,
        0.42038748, -0.0635958 , -0.09683355,  0.18086288,  0.23704431,
        0.01412683,  0.1700973 , -1.14917018,  0.31498588,  0.06622261])

In [53]:
df_glove.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,...,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
",",-0.082752,0.672040,-0.14987,-0.064983,0.056491,0.402280,0.002775,-0.331100,-0.306910,2.0817,0.031819,0.013643,0.30265,0.007130,-0.58190,...,0.074901,0.061068,-0.466200,0.400540,-0.190990,-0.14331,0.018267,-0.18643,0.207090,-0.355980,0.053380,-0.050821,-0.191800,-0.378460,-0.06589
.,0.012001,0.207510,-0.12578,-0.593250,0.125250,0.159750,0.137480,-0.331570,-0.136940,1.7893,-0.470940,0.704340,0.26673,-0.089961,-0.18168,...,0.021307,-0.107780,-0.228100,0.508030,0.115670,0.16165,-0.066737,-0.29556,0.022612,-0.281350,0.063500,0.140190,0.138710,-0.360490,-0.03500
the,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.5882,-0.351790,-0.173120,0.43285,-0.107080,0.15006,...,-0.301930,0.043579,-0.043102,0.350250,-0.196810,-0.42810,0.168990,0.22511,-0.285570,-0.102800,-0.018168,0.114070,0.130150,-0.183170,0.13230
and,-0.185670,0.066008,-0.25209,-0.117250,0.265130,0.064908,0.122910,-0.093979,0.024321,2.4926,-0.017916,-0.071218,-0.24782,-0.262370,-0.22460,...,-0.005455,0.477960,0.090912,0.094489,-0.368820,-0.59396,-0.097729,0.20072,0.170550,-0.004736,-0.039709,0.324980,-0.023452,0.123020,0.33120
to,0.319240,0.063160,-0.27858,0.261200,0.079248,-0.214620,-0.104950,0.154950,-0.033530,2.4834,-0.509040,0.087490,0.21426,0.221510,-0.25234,...,-0.237020,0.038399,-0.100310,0.183590,0.025178,-0.12977,0.371300,0.18888,-0.004274,-0.106450,-0.258100,-0.044629,0.082745,0.097801,0.25045


In [54]:
# Creating a vectorize representation for each job title in our dataframe
job_titles = df.job_title_cleaned

doc_sent_vec = []

for sentences in job_titles:
    word_vec = []
    for word in sentences.split():
        if word in glove:
            vectors = glove[word]
            word_vec.append(vectors)
        else:
            word_vec.append(unknown_word)
    word_vec_mean = sum(word_vec) / len(word_vec) # returning a mean for each job title
    doc_sent_vec.append(word_vec_mean) # returning a list for all job titles

In [55]:
len(doc_sent_vec)

53

In [56]:
doc_sent_vec[0].shape

(300,)

In [57]:
# Creating a vectorize representation for each query
def q_sent_vec(query):
    q_sent_vec = []
    q_word_vec = []
    
    for word in query.split():
        if word in glove:
            vectors = glove[word]
            q_word_vec.append(vectors)
        else:
            q_word_vec.append(unknown_word)
        q_word_vec_mean = sum(q_word_vec) / len(q_word_vec)
    q_sent_vec.append(q_word_vec_mean)
        
    return q_sent_vec

In [58]:
query = 'native english speaking'
len(q_sent_vec(query))

1

In [59]:
q_sent_vec(query)[0].shape

(300,)

In [60]:
q_sent_vec(query)[0][:5]

array([-0.29654333,  0.12640833, -0.49922333,  0.22307667,  0.4358    ])

In [61]:
query = 'student indiana university'
q_sent_vec(query)[0][:5]

array([-0.10656   ,  0.06428367,  0.10134093, -0.19890667,  0.51552   ])

In [62]:
def get_glove_query_similarity(doc_sent_vec, query):
    """
    query_glove: processing the query
    doc_sent_vec: glove embedding for all docs
    query: query doc

    return: cosine similarity between query and all docs

    """
    query_glove = q_sent_vec(query)
    
    cos_sim_glove = cosine_similarity(query_glove, doc_sent_vec).flatten()
    
    return cos_sim_glove

In [63]:
query_glove = 0

In [64]:
query = 'Aspiring human resources'

#GloVe similarity
cos_sim_glove = get_glove_query_similarity(doc_sent_vec, query = query)
df['glove_fit'] = cos_sim_glove

# original TFIDF similarity and Word2Vec Similarity for comparison
cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query) 
df['tfidf_fit'] = cos_sim

cos_sim_w2v = get_w2v_query_similarity(document_word_embeddings, query = query)
df['w2v_fit'] = cos_sim_w2v

In [65]:
top_candidates(n = 10, by = 'glove_fit', ascending = False, min_con = 0)

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit,glove_fit
id,,,,,,,
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,0.898174,0.735855,0.851023
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,0.898174,0.735855,0.851023
6,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,0.873679,0.632697,0.848638
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seeking inte...,0.584569,0.508880,0.845360
74,Human Resources Professional,Greater Boston Area,16,human resources professional,0.134220,0.340769,0.836803
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,seeking human resources opportunities,0.619797,0.220668,0.825179
101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",501,human resources generalist loparex,0.202520,0.196509,0.799749
68,Human Resources Specialist at Luxottica,Greater New York City Area,501,human resources specialist luxottica,0.151158,0.189503,0.790386
99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,seeking human resources position,0.654387,0.220668,0.776370


In [66]:
query = 'seeking human resources'

#GloVe similarity
cos_sim_glove = get_glove_query_similarity(doc_sent_vec, query = query)
df['glove_fit'] = cos_sim_glove

# original TFIDF similarity and Word2Vec Similarity for comparison
cos_sim = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query = query) 
df['tfidf_fit'] = cos_sim

cos_sim_w2v = get_w2v_query_similarity(document_word_embeddings, query = query)
df['w2v_fit'] = cos_sim_w2v

In [67]:
top_candidates(n = 10, by = 'glove_fit', ascending = False, min_con = 0)

,job_title,location,connection,job_title_cleaned,w2v_fit,tfidf_fit,glove_fit
id,,,,,,,
28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,seeking human resources opportunities,0.839381,0.675682,0.970024
99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,seeking human resources position,0.886226,0.675682,0.953714
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seeking inte...,0.431644,0.362648,0.935586
74,Human Resources Professional,Greater Boston Area,16,human resources professional,0.133104,0.295223,0.903558
94,Seeking Human Resources Opportunities. Open t...,Amerika Birleşik Devletleri,415,seeking human resources opportunities open tr...,0.639099,0.381290,0.885495
6,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,0.645122,0.206629,0.874185
100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,aspiring human resources manager graduating ...,0.343832,0.220083,0.870053
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,0.663209,0.240319,0.864091
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,0.663209,0.240319,0.864091


# Fasttext - 

In [68]:
# Downloading fastText pre-trained vectors
# wget.download('https://github.com/facebookresearch/fastText/archive/v0.9.2.zip')

In [69]:
# Extracting fastText vector file
# import zipfile as zf
# files = zf.ZipFile("fastText-0.9.2.zip", 'r')
# files.extractall()
# files.close()

In [70]:
import os
os.getcwd()

'C:\\Users\\Administrator\\Documents\\the-Lab\\Apziva\\tirMlkapx4XpMRlG'

In [71]:
os.chdir(r'C:\Users\Administrator\Documents\the-Lab\Apziva\tirMlkapx4XpMRlG\fastText-0.9.2')

In [72]:
os.getcwd()

'C:\\Users\\Administrator\\Documents\\the-Lab\\Apziva\\tirMlkapx4XpMRlG\\fastText-0.9.2'

#### Issues and workarounds with installing fasttext:

https://stackoverflow.com/questions/44951456/pip-error-microsoft-visual-c-14-0-is-required

In [73]:
# !pip install --upgrade pip
# !pip install --upgrade wheel
# !pip install --upgrade setuptools
# !pip install Cython --install-option="--no-cython-compile"

In [76]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-win_amd64.whl size=246190 sha256=554b001eae8c043207aa6fe0038dd34122370fbc5f9f0f48c21f97bc2b1ddb36
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\4e\ca\bf\b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [88]:
import fasttext as ft

In [82]:
text = df.job_title_cleaned

In [86]:
model = ft.train_unsupervised(text, model='skipgram')

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.args, arg0: str) -> None

Invoked with: <fasttext_pybind.args object at 0x000001AD82989630>, id
1           c t  bauer college business graduate  mag...
2      native english teacher epik  english program k...
3                  aspiring human resources professional
4                    people development coordinator ryan
5           advisory board member celal bayar university
6                    aspiring human resources specialist
7      student humber college aspiring human resource...
8                                   hr senior specialist
10     seeking human resources hris generalist positions
11                            student chapman university
12     svp  chro  marketing   communications  csr off...
13     human resources coordinator intercontinental b...
27     aspiring human resources management student se...
28                 seeking human resource

In [81]:
# model = fasttext.train_supervised(input=df.job_title_cleaned)

model = fasttext.train_unsupervised('df.job_title_cleaned', model='skipgram')

ValueError: df.job_title_cleaned cannot be opened for training!

In [87]:
sent_vectors=[]
for line in df['job_title_cleaned']:
    vectors=model.get_sentence_vector(line, k=-1, threshold=0.5)[0][0]
    sent_vectors.append(vectors)



NameError: name 'model' is not defined

In [ ]:
df['ft_fit']=sent_vectors

In [ ]:
import ktrain
from ktrain import text

In [ ]:
NUM_WORDS = 50000
MAXLEN = 150

In [ ]:
text.tests_from

# BERT - 

In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text

In [ ]:
# Input candidates, query term, location, etc

In [ ]:
# WordtoVec  Same thing but with pretrained word embedding average of word
# Try to see who I'm connected with 
skill review surrvey - schedule interview - motivated 